In [8]:
%reset -f
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [9]:
#df = pd.read_csv("SDS00002.CSV", skiprows=[1])
df = pd.read_csv("SDS00001.CSV", skiprows=[1])
df.describe()

Rhi = 4.7e6*5
Rlo = 110e3
G = (Rhi+Rlo)/Rlo
G *= 1.14

# Assign decent names to the columns and compute derivatives
df["t"] = df.Source - df.Source.min()
df["u_in_gt"] = df.CH4
df["u_out"] = df.CH1 * G
# Remove the silly-named original columns
del df["Source"]
del df["CH1"]
del df["CH4"]

desired_sample_rate = 1e3
time_span = df.t.max() - df.t.min()
desired_num_samples = int(time_span * desired_sample_rate)
downsample_factor = round(len(df) / desired_num_samples)
print(f"{downsample_factor=}")
num_rows = (len(df) // downsample_factor) * downsample_factor
df = df.iloc[:num_rows]
reshaped_df = df.values.reshape(-1, downsample_factor, df.shape[1])
downsampled_data = reshaped_df.mean(axis=1)
dsdf = pd.DataFrame(downsampled_data, columns=df.columns)
dsdf["dt"] = dsdf.t.diff()
dsdf["du_out_dt"] = dsdf.u_out.diff() / dsdf.dt
dsdf["du_out_dt_lpf"] = dsdf.du_out_dt.rolling(5).mean()

# Trim the dataframe to only include the part where the cap is being charged
begin_at = dsdf[dsdf.du_out_dt_lpf > +1000].index[0] - 1
dsdf     = dsdf.iloc[begin_at:]
end_at   = dsdf[dsdf.du_out_dt_lpf < -1000].index[0] - 1
dsdf     = dsdf.iloc[:end_at-begin_at]
dsdf

# Trim the time base.
dsdf["t"] = dsdf.t - dsdf.t.min()
dsdf.describe()

downsample_factor=400


,t,u_in_gt,u_out,dt,du_out_dt,du_out_dt_lpf
count,1429.000000,1429.000000,1429.000000,1.429000e+03,1429.000000,1429.000000
mean,0.714000,575.591987,576.718274,1.000000e-03,541.458603,540.917522
std,0.412661,173.673872,170.461375,4.681321e-16,2358.837940,959.124578
min,0.000000,-0.275000,2.887288,1.000000e-03,-6410.758909,-920.017309
25%,0.357000,472.800000,478.604749,1.000000e-03,-978.741818,19.574836
50%,0.714000,624.225000,628.792681,1.000000e-03,685.119273,430.646400
75%,1.071000,717.425000,713.698534,1.000000e-03,2153.232000,871.080218
max,1.428000,780.225000,777.610375,1.000000e-03,36115.573091,13829.621891


In [10]:
R_eq = (Rhi * Rlo) / (Rhi + Rlo)
C_out = 1e-9
tau_init = C_out * R_eq
tau_init

0.0001094875052943668

In [11]:
def predict_u_in(u_out, du_out_dt, tau):
    return u_out + tau * du_out_dt

dsdf["u_in_pred"] = predict_u_in(dsdf.u_out, dsdf.du_out_dt_lpf, tau_init)

fig = go.Figure()
fig.update_layout(
    height=400,
    hovermode='x',
    yaxis2=dict(title='u_c [V]', overlaying='y', side='right'),
)
fig.add_trace(go.Scattergl(x=dsdf.t, y=dsdf.u_in_gt, name="u_in_gt"))
fig.add_trace(go.Scattergl(x=dsdf.t, y=dsdf.u_out, name="u_out"))
fig.add_trace(go.Scattergl(x=dsdf.t, y=dsdf.u_in_pred, name="u_in_pred"))
fig.add_trace(go.Scattergl(x=dsdf.t, y=dsdf.du_out_dt_lpf, name="du_out/dt", yaxis='y2'))

In [12]:
fig = go.Figure()
fig.update_layout(height=400, hovermode='x')
fig.add_trace(go.Scattergl(x=dsdf.t, y=dsdf.u_in_pred-dsdf.u_in_gt, name="u_err"))